In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks
import numpy as np
from tensorboard.plugins.hparams import api as hp

2024-03-27 22:02:31.571432: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 22:02:31.571487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 22:02:31.572813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-27 22:02:31.580868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 22:02:32.534331: W tensorflow/compiler/tf2

In [13]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__(name="test1")
        self.folder = "./treino/test"
        self.dense1 = tf.keras.layers.Dense(1, activation=tf.nn.relu, input_shape=(None,1), name="1")
        self.dense2 = tf.keras.layers.Dense(2, activation=tf.nn.relu, input_shape=(None,2), name="2")
        self.dense3 = tf.keras.layers.Dense(6, activation=tf.nn.relu, name="3")
        self.dense4 = tf.keras.layers.Dense(2, activation=tf.nn.relu, name="4")
    
    def tensorboard_writer(self):
        return tf.summary.create_file_writer("./tensorboard/fit")
    
    def write_to_tensorboard(self, value, step):
        with self.tensorboard_writer().as_default():
            tf.summary.scalar('random', value, step=step)
        # test_accuracy.reset_states()

    def call(self, X): #do not use
        (a,b) = X
        print("call")
        a = self.dense1(a)
        c = tf.concat([a, tf.cast(tf.convert_to_tensor(b), tf.float32)],1)
        c = self.dense2(c)
        c = self.dense3(c)

        return self.dense4(c)

    def train(self, X, y, validation_data, shuffle, epocs, batch, lr):
        self.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
            loss=tf.keras.losses.MeanSquaredError()
        )
        print("train")
        self.history = self.fit(
                X,
                y,
                validation_data=validation_data, 
                shuffle=shuffle, 
                epochs= epocs,
                batch_size= batch,
                verbose= 1,
                callbacks = [
                    callbacks.EarlyStopping(monitor="val_loss", patience=2),
                    callbacks.ModelCheckpoint(self.folder, save_best_only = True),
                    callbacks.TensorBoard(
                                    log_dir="./tensorboard/fit",
                                    histogram_freq=1,
                                    write_graph=True,
                                    write_images=False,
                                    write_steps_per_second=False,
                                    update_freq="epoch",
                                    profile_batch=1,
                                    embeddings_freq=0,
                                    embeddings_metadata=None
                                )
                ]
            )


model = MyModel()


In [14]:
model((np.asarray([[1],[2]]), np.asarray([[1],[2]])))

call


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.6012084 , 0.84330153],
       [1.2024168 , 1.6866031 ]], dtype=float32)>

In [15]:
model.summary()

Model: "test1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 1 (Dense)                   multiple                  2         
                                                                 
 2 (Dense)                   multiple                  6         
                                                                 
 3 (Dense)                   multiple                  18        
                                                                 
 4 (Dense)                   multiple                  14        
                                                                 
Total params: 40 (160.00 Byte)
Trainable params: 40 (160.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.write_to_tensorboard(22, 11)

In [17]:
tf.convert_to_tensor( np.asarray([[[0,1],[0,1]]])[0] )

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[0, 1],
       [0, 1]])>

In [18]:
model.train(
        X=(np.asarray([[1],[2],[3],[4],[5],[6],[1],[2],[3],[4],[5],[6]]), np.asarray([[1],[2],[3],[4],[5],[6],[6],[1],[2],[3],[4],[5]])),
        y=( np.asarray([[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1]]) ),
        shuffle=True,
        validation_data = (
            (np.asarray([[1],[2],[3],[4],[5],[6],[1],[2],[3],[4],[5],[6]]), np.asarray([[1],[2],[3],[4],[5],[6],[6],[1],[2],[3],[4],[5]])),
            np.asarray([[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1]])
        ),
        epocs=50,
        batch=3,
        lr=0.1
    )

train
Epoch 1/50
call


2024-03-27 22:03:58.996980: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-27 22:03:58.997021: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-03-27 22:03:58.997052: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:135] cuptiGetTimestamp: ignored due to a previous error.
2024-03-27 22:03:58.997070: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:186] cuptiSubscribe: ignored due to a previous error.
2024-03-27 22:03:58.997080: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:459] cuptiGetResultString: ignored due to a previous error.
2024-03-27 22:03:58.997090: E external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1935] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2024-03-27 22:03:58.998106: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131]

call
1/4 [======>.......................] - ETA: 3s - loss: 8.5450

2024-03-27 22:04:00.165666: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-03-27 22:04:00.169179: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:142] cuptiFinalize: ignored due to a previous error.
2024-03-27 22:04:00.169213: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:459] cuptiGetResultString: ignored due to a previous error.
2024-03-27 22:04:00.169223: E external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:2026] function cupti_interface_->Finalize()failed with error 
2024-03-27 22:04:00.173714: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:135] cuptiGetTimestamp: ignored due to a previous error.
2024-03-27 22:04:00.173752: E external/local_xla/xla/backends/profiler/gpu/cupti_error_manager.cc:135] cuptiGetTimestamp: ignored due to a previous error.
2024-03-27 22:04:00.173764: I external/local_xla/xla/backends/profiler/gpu/cupti_collector.cc:541]  GpuT

call
call
call


In [ ]:
%tensorboard --logdir ./tensorboard/fit

Reusing TensorBoard on port 6006 (pid 21865), started 0:00:20 ago. (Use '!kill 21865' to kill it.)